In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mattpy.utils import smooth

In [2]:
def save_shift_figure(plot_dir, classifier, iso_filename, wave, flux,
                      smooth_flux, smooth_flux_shift, verbose=True):
    """Saves a .PDF figure of the raw spectrum and shifted smoothed spectrum."""
    
    # Plot the raw spectrum and smoothed spectrum.
    plt.plot(wave, flux)
    plt.plot(wave, smooth_flux)
    plt.axhline(y=0, color='k', ls='--', lw=1)
    plt.axhline(y=np.nanmax(smooth_flux_shift), ls='--', lw=1,
                zorder=-10, color='k')
    
    # Save to disk.
    plt.title(iso_filename + ' - classifier: ' + classifier)
    savepath = plot_dir + iso_filename + '_A.pdf'
    plt.savefig(savepath, format='pdf', bbox_inches='tight')
    plt.close()
    
    if verbose:
        print('Saved: ', savepath)
    
    return


def save_renorm_figure(plot_dir, classifier, iso_filename, renorm_wave,
                       renorm_flux_shift, smooth_flux_shift,
                       verbose=True):
    """Saves a .PDF figure of the renormalized spectrum."""
    
    # Plot the renormalized/shifted spectrum.
    plt.plot(renorm_wave, renorm_flux_shift)
    plt.plot(renorm_wave, smooth_flux_shift)
    plt.axhline(y=0, color='red', ls='--', lw=1)
    plt.axhline(y=1, ls='--', lw=1, zorder=-10, color='red')
    
    # Save to disk.
    plt.title(iso_filename + ' - classifier: ' + classifier)
    savepath = plot_dir + iso_filename + '_B.pdf'
    plt.savefig(savepath, format='pdf', bbox_inches='tight')
    plt.close()
    
    if verbose:
        print('Saved: ', savepath)
    
    return

In [3]:
def read_spectrum(file_path):
    """Returns an ISO spectrum (wave, flux, etc.) from a pickle."""
    spectrum = pd.read_pickle(file_path)
    
    wave = spectrum['wavelength']
    flux = spectrum['flux']
#     specerr = spectrum['spec_error']
#     normerr = spectrum['norm_error']
#     fluxerr = specerr + normerr
    fluxerr = spectrum['uncertainty']
    
    return wave, flux, fluxerr


def smooth_spectrum(flux, **kwargs):
    """Returns a shifted, smoothed spectrum, ready for normalization."""
    spec_min = np.nanmin(flux)
    spec_max = np.nanmax(flux)
    # print(spec_min, spec_max)

    # Smooth it to find the general low/high points.    
    smooth_flux = smooth(flux.values, **kwargs)
    
    # Smooth it to find the general low/high points, and shift to zero.
    smooth_flux_shift = smooth(flux.values - spec_min, **kwargs)

    # Upper normalization factor (s.t. the maximum of the continuum is 1.0).
    norm_factor = np.nanmax(smooth_flux_shift)

    return spec_min, spec_max, smooth_flux, smooth_flux_shift, norm_factor


def normalize_spectrum(file_path, classifier, plot=True, verbose=True):
    """Normalizes an ISO spectrum to span 0-1 (the main curvature)."""
    wave, flux, fluxerr = read_spectrum('../../' + file_path)
    
    # Shift spectrum and get normalization factors.
    # Minimum should now=0.0.
    spec_min, spec_max, smooth_flux, smooth_flux_shift, norm_factor = \
        smooth_spectrum(flux, window_len=40)

    # Final renormalized quantities.
    renorm_wave = wave
    renorm_flux_shift = (flux - spec_min) / norm_factor
    
    # Plotting directory
    plot_dir = 'plots/'
    
    # Save file name.
    iso_filename = file_path.split('/')[-1].split('.pkl')[0]
    
    if plot:
        # Save a figure showing the initial smooth/shift.
        save_shift_figure(plot_dir, classifier, iso_filename,
                          wave, flux, smooth_flux, smooth_flux_shift,
                          verbose=False)

        # Save a figure of the final renormalized spectrum.
        save_renorm_figure(plot_dir, classifier, iso_filename,
                           renorm_wave, renorm_flux_shift,
                           smooth_flux_shift/norm_factor,
                           verbose=False)
    
    return spec_min, spec_max, norm_factor

In [4]:
meta = pd.read_pickle('../metadata_step0.pkl')

In [5]:
nrows = meta.shape[0]
meta

object_name       tdt          ra        dec  \
0                    NGC 6543   2400714  269.639167  66.633194   
1                    NGC 6543   2400807  269.639167  66.633194   
2                    NGC 6543   2400910  269.639125  66.633194   
3                    NGC 7027   2401183  316.757125  42.235861   
4                 {gamma} Dra   2401579  269.151708  51.488972   
5                 {gamma} Dra   2402105  269.151667  51.488972   
6                   AFGL 2591   2800433  307.352750  40.188583   
7                    NGC 6543   2800908  269.639167  66.633194   
8                       P Cyg   3201129  304.446667  38.032861   
9                    NGC 6543   3201202  269.639167  66.633194   
10             Cyg OB2 No. 12   3602226  308.170833  41.241472   
11                    HR 4049   4000458  154.531667 -28.992028   
12                     AG Car   4000652  164.048333 -60.453722   
13                   V778 Cyg   4001456  309.044167  60.088889   
14                 {beta} Leo   4001710  177.263667  14.572194   
15                {gamma} Dra   4002405  269.151708  51.488972   
16                    NML Cyg   5200726  311.606250  40.116500   
17    {alpha} Boo +0.04, +1.0   5601291  213.916708  19.184611   
18            IRAS 21282+5050   5602477  322.493417  51.066611   
19                 {beta} Peg   5602595  345.943958  28.082667   
20                   {mu} Cep   5602852  325.876667  58.780028   
21                  AFGL 4106   6000280  155.831875 -59.534944   
22                    HR 7341   6001362  289.657458  49.569333   
23                      S 140   6401081  334.826250  63.313083   
24                   {xi} Dra   6800697  268.381958  56.872583   
25                  {eta} Car   7100250  161.265000 -59.684528   
26    {alpha} Boo +0.03, +0.9   7100434  213.916667  19.184583   
27                  HD 161796   7100579  266.231250  50.044389   
28                {delta} Cen   7200272  182.089625 -50.722417   
29                {delta} Cen   7200351  182.089625 -50.722417   
...                       ...       ...         ...        ...   
1205                AFGL 2392  85800120  291.810000   7.069444   
1206          Lk H{alpha} 225  85800403  305.126667  41.357500   
1207          Lk H{alpha} 224  85800502  305.121667  41.357500   
1208                    W Ori  85801604   76.348750   1.177556   
1209               IRC +50137  86201803   77.831042  52.876028   
1210          IRAS 04579+4703  86201902   75.415417  47.123111   
1211                   RV Cam  86202101   67.673750  57.411611   
1212                 AFGL 437  86300810   46.848667  58.514056   
1213                AFGL 4029  86300968   45.380375  60.487083   
1214                   SZ Sgr  86400616  266.235417 -18.657500   
1215               BD+30 3639  86500540  293.688333  30.516333   
1216              {alpha} Tuc  86602401  334.625500 -60.259694   
1217                  HR 1939  86603434   85.175000  31.920833   
1218               IRC -10122  86706617   96.256667  -9.121111   
1219               IRC -10095  86801101   84.927500  -8.152194   
1220              {alpha} CMa  86801303  101.287375 -16.715528   
1221              OH 26.2-0.6  86901013  280.307500  -6.250278   
1222          IRAS 17347-3139  87000939  264.502583 -31.681722   
1223                 AFGL 940  87102602   96.655417   9.037778   
1224               IRC +40149  87102906   89.853333  38.439444   
1225          IRAS 17074-4549  87200109  257.777583 -45.883194   
1226              OH 21.5+0.5  87200833  277.128750  -9.971111   
1227               IRC +10374  87201107  280.893750  13.959722   
1228                 Case 181  87201221  285.825417   7.512222   
1229                    T Mic  87201305  306.980000 -28.261111   
1230               IRC +00102  87201709   95.463750  -3.861667   
1231                   FU Ori  87601701   86.343750   9.070000   
1232                     M 16  87700401  274.735833 -13.814583   
1233                   FI Vul  87700716  312.213333  22.99

In [6]:
def norm_and_plot(meta):
    param_list = []
    
    for index, filename in enumerate(meta['file_path']):
        if index % 200 == 0:
            print(index, ' / ', nrows)

        # Full classifier
        classifier = meta['full_classifier'][index]
            
        # Perform shift/renormalization
        parameters = normalize_spectrum(filename, classifier,
                                        plot=True, verbose=False)
        
        # Save parameters to a list
        spec_min, spec_max, norm_factor = parameters
        param_list.append([filename, *parameters])
        
    return param_list

In [7]:
par_list = norm_and_plot(meta)

0  /  1235
200  /  1235
400  /  1235
600  /  1235
800  /  1235
1000  /  1235
1200  /  1235


In [8]:
np.savetxt('step1_norm_params.txt', par_list, delimiter=',', fmt='%s',
           header='iso_filename, spec_min, spec_max, norm_factor (shift first, then norm!!)')